****
## JSON Mini Project

Using data in file 'data/world_bank_projects.json':
1. Find the 10 countries with most projects
2. Find the top 10 major project themes (using column 'mjtheme_namecode')
3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

In [156]:
# load json as string
WBproj = json.load((open('data/world_bank_projects.json')))
WBproj;

In [164]:
# load as Pandas dataframe
df_WBproj = pd.read_json('data/world_bank_projects.json')
df_WBproj.head()

,_id,approvalfy,board_approval_month,boardapprovaldate,borrower,closingdate,country_namecode,countrycode,countryname,countryshortname,...,sectorcode,source,status,supplementprojectflg,theme1,theme_namecode,themecode,totalamt,totalcommamt,url
0,{'$oid': '52b213b38594d8a2be17c780'},1999,November,2013-11-12T00:00:00Z,FEDERAL DEMOCRATIC REPUBLIC OF ETHIOPIA,2018-07-07T00:00:00Z,Federal Democratic Republic of Ethiopia!$!ET,ET,Federal Democratic Republic of Ethiopia,Ethiopia,...,"ET,BS,ES,EP",IBRD,Active,N,"{'Percent': 100, 'Name': 'Education for all'}","[{'code': '65', 'name': 'Education for all'}]",65,130000000,130000000,http://www.worldbank.org/projects/P129828/ethi...
1,{'$oid': '52b213b38594d8a2be17c781'},2015,November,2013-11-04T00:00:00Z,GOVERNMENT OF TUNISIA,NaN,Republic of Tunisia!$!TN,TN,Republic of Tunisia,Tunisia,...,"BZ,BS",IBRD,Active,N,"{'Percent': 30, 'Name': 'Other economic manage...","[{'code': '24', 'name': 'Other economic manage...","54,24",0,4700000,http://www.worldbank.org/projects/P144674?lang=en
2,{'$oid': '52b213b38594d8a2be17c782'},2014,November,2013-11-01T00:00:00Z,MINISTRY OF FINANCE AND ECONOMIC DEVEL,NaN,Tuvalu!$!TV,TV,Tuvalu,Tuvalu,...,TI,IBRD,Active,Y,"{'Percent': 46, 'Name': 'Regional integration'}","[{'code': '47', 'name': 'Regional integration'...","52,81,25,47",6060000,6060000,http://www.worldbank.org/projects/P145310?lang=en
3,{'$oid': '52b213b38594d8a2be17c783'},2014,October,2013-10-31T00:00:00Z,MIN. OF PLANNING AND INT'L COOPERATION,NaN,Republic of Yemen!$!RY,RY,Republic of Yemen,"Yemen, Republic of",...,JB,IBRD,Active,N,"{'Percent': 50, 'Name': 'Participation and civ...","[{'code': '57', 'name': 'Participation and civ...","59,57",0,1500000,http://www.worldbank.org/projects/P144665?lang=en
4,{'$oid': '52b213b38594d8a2be17c784'},2014,October,2013-10-31T00:00:00Z,MINISTRY OF FINANCE,2019-04-30T00:00:00Z,Kingdom of Lesotho!$!LS,LS,Kingdom of Lesotho,Lesotho,...,"FH,YW,YZ",IBRD,Active,N,"{'Percent': 30, 'Name': 'Export development an...","[{'code': '45', 'name': 'Export development an...","41,45",13100000,13100000,http://www.worldbank.org/projects/P144933/seco...


In [158]:
# 1. Find the 10 countries with most projects

# print the first 10 entries of value_counts() on 'countryname' 
print(df_WBproj.countryname.value_counts()[0:10])

Republic of Indonesia              19
People's Republic of China         19
Socialist Republic of Vietnam      17
Republic of India                  16
Republic of Yemen                  13
People's Republic of Bangladesh    12
Kingdom of Morocco                 12
Nepal                              12
Africa                             11
Republic of Mozambique             11
Name: countryname, dtype: int64


In [159]:
# 2. Find the top 10 major project themes (using column 'mjtheme_namecode')

#assuming that the major theme is limited to only the first listed theme for each project:

# create table using json normalization
# keep only the first listed theme per project id
df_themes = json_normalize(WBproj, 'mjtheme_namecode', ['id'])
df_themes = df_themes.groupby('id').first()

# print the first 10 entries of value_counts() on 'name' 
print(df_themes.name.value_counts()[0:10])

Environment and natural resources management    85
Human development                               72
Public sector governance                        64
Social protection and risk management           57
Rural development                               56
Financial and private sector development        53
Social dev/gender/inclusion                     43
Trade and integration                           25
Urban development                               23
Economic management                             11
Name: name, dtype: int64


In [170]:
# 3. Create a dataframe with the missing names filled in.

# create dictionary of codes to names
df_theme_dict = df_themes.groupby('code').name.max()
theme_dict = df_theme_dict.to_dict()

print(theme_dict)

{'1': 'Economic management', '10': 'Rural development', '11': 'Environment and natural resources management', '2': 'Public sector governance', '3': 'Rule of law', '4': 'Financial and private sector development', '5': 'Trade and integration', '6': 'Social protection and risk management', '7': 'Social dev/gender/inclusion', '8': 'Human development', '9': 'Urban development'}


In [171]:
# fill in missing values using theme_dict
for a in df_WBproj.mjtheme_namecode:
    for b in a:
        if b['name'] == '':
            b['name'] = theme_dict[b['code']]

# check if missing values were filled in; if code was incorrect it will print 'not successful'
for a in df_WBproj.mjtheme_namecode:
    for b in a:
        if b['name'] == '':
            print('not successful')